In [2]:
import os
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import base64
import numpy as np
import io
from skimage import io as skio, transform

In [3]:
model = tf.keras.models.load_model('C:/Users/Adam/Desktop/WTUM/snn_model.h5')

In [4]:
app = dash.Dash(__name__)

ascii_mapping = {
    0: "#",   1: "$",   2: "&",   3: "0",   4: "1",   5: "2",   6: "3",   7: "4",
    8: "5",   9: "6",  10: "7",  11: "8",  12: "9",  13: "@",  14: "A",  15: "B",
   16: "C",  17: "D",  18: "E",  19: "F",  20: "G",  21: "H",  22: "I",  23: "J",
   24: "K",  25: "L",  26: "M",  27: "N",  28: "P",  29: "Q",  30: "R",  31: "S",
   32: "T",  33: "U",  34: "V",  35: "W",  36: "X",  37: "Y",  38: "Z"
}

def predict_image(img):
    image = skio.imread(io.BytesIO(base64.b64decode(img.split(',')[1])))
    image = transform.resize(image, (28, 28))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0

    prediction = model.predict(image)
    predicted_label = np.argmax(prediction) 

    predicted_label = ascii_mapping.get(predicted_label, "nieznany")
    return predicted_label

app.layout = html.Div([
    html.H1("EMNIST rozpoznawanie znaków", style={'color': 'white'}),
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Przęciągnij i upuść lub ',
            html.A('Wybierz z plików obraz', style={'color': 'white'})
        ]),
        style={
            'width': '50%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px',
            'color': 'white',
            'borderColor': 'white'
        },
        multiple=False
    ),
    html.Div(id='output-image-upload', style={'color': 'white'}),
], style={'backgroundColor': 'black', 'height': '100vh', 'padding': '10px'})

@app.callback(Output('output-image-upload', 'children'),
              [Input('upload-image', 'contents')],
              [State('upload-image', 'filename')])
def update_output(contents, filename):
    if contents is not None:
        return html.Div([
            html.H5(filename),
            html.Img(src=contents, style={'height': '200px'}),
            html.Hr(),
            html.H6(f'Predicted Label: {predict_image(contents)}')
        ])

if __name__ == '__main__':
    app.run_server(debug=True, port=8055)